# Rwanda Aurora - GPU Training on Kaggle\n\nThis notebook trains the Rwanda-specific Aurora weather forecasting model on Kaggle's GPU.\n\n**Setup:**\n1. Enable GPU: Settings → Accelerator → GPU T4 x2\n2. Upload your ERA5 data as a Kaggle dataset OR use synthetic data\n3. Run all cells\n\n**Expected Training Time:** ~10-30 minutes on GPU

In [ ]:
# Install required packages\n!pip install -q wandb einops timm xarray netcdf4\n!pip install -q seaborn matplotlib scikit-learn

In [ ]:
# Import training script\nimport sys\nimport torch\nimport numpy as np\nsys.path.append('/kaggle/working')\n\n# Check GPU availability\nprint(f\"PyTorch version: {torch.__version__}\")\nprint(f\"CUDA available: {torch.cuda.is_available()}\")\nif torch.cuda.is_available():\n    print(f\"GPU: {torch.cuda.get_device_name(0)}\")\n    print(f\"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB\")\nelse:\n    print(\"⚠️  No GPU detected. Training will be slow!\")\n    print(\"Go to: Settings → Accelerator → GPU T4 x2\")

In [ ]:
# Clone repository or upload files\nimport os\n\nif not os.path.exists('Aurora'):\n    !git clone https://github.com/AlainKwishima/Aurora.git\n    %cd Aurora\nelse:\n    %cd Aurora\n    !git pull

In [ ]:
# Install Aurora package\n!pip install -e .

In [ ]:
# Configure training for GPU\nfrom notebooks.rwanda_aurora_training import KaggleConfig\n\nclass GPUKaggleConfig(KaggleConfig):\n    \"\"\"Optimized config for Kaggle GPU\"\"\"\n    \n    @staticmethod\n    def get_device():\n        if torch.cuda.is_available():\n            return torch.device('cuda')\n        return torch.device('cpu')\n    \n    @staticmethod\n    def training_config():\n        return {\n            'batch_size': 8,  # Larger batch with GPU\n            'num_epochs': 100,\n            'learning_rate': 3e-5,\n            'accumulation_steps': 2,  # Effective batch: 16\n            'save_every': 5,\n            'val_check_interval': 10,\n            'early_stopping_patience': 20,\n            'gradient_clip_val': 1.0,\n        }\n        \n# Override config\nKaggleConfig = GPUKaggleConfig

In [ ]:
# Optional: Initialize Weights & Biases for tracking\ntry:\n    import wandb\n    wandb.login()\n    wandb.init(project=\"rwanda-aurora\", config=KaggleConfig.training_config())\n    use_wandb = True\nexcept:\n    print(\"W&B not configured. Skipping...\")\n    use_wandb = False

In [ ]:
# Check for data\nimport os\n\n# Look for uploaded ERA5 data\ndata_paths = [\n    '/kaggle/input/rwanda-era5/rwanda_era5.nc',\n    '/kaggle/input/rwanda-era5-train/rwanda_era5_train.nc',\n    'data/rwanda_era5.nc',\n]\n\ndata_file = None\nfor path in data_paths:\n    if os.path.exists(path):\n        data_file = path\n        print(f\"✓ Found data: {path}\")\n        break\n\nif not data_file:\n    print(\"⚠️  No ERA5 data found. Will generate synthetic data.\")\n    print(\"For real training, upload ERA5 data as a Kaggle dataset.\")

In [ ]:
# Run training\nfrom notebooks.rwanda_aurora_training import main\n\ntry:\n    main()\nexcept Exception as e:\n    print(f\"❌ Training failed: {e}\")\n    import traceback\n    traceback.print_exc()\nfinally:\n    if use_wandb:\n        wandb.finish()

In [ ]:
# Download trained model\nfrom IPython.display import FileLink\n\nprint(\"Download your trained model:\")\ndisplay(FileLink('working/best_rwanda_aurora.pt'))\ndisplay(FileLink('working/training_history.png'))

## Next Steps\n\n1. Download the trained model (`best_rwanda_aurora.pt`)\n2. Run evaluation: `python evaluate_model.py`\n3. Test inference: `python quick_inference.py`